# Vector Stores

## Introduction

In [89]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_model = OllamaEmbeddings(model='snowflake-arctic-embed:22m')


## Basics of Vector Stores

In [90]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

# Initialize with an embedding model
vector_store = InMemoryVectorStore(embedding=embedding_model)

# Create a document
document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "news"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

documents = [document_1, document_2]
# Add documents
vector_store.add_documents(documents=documents)
# Add documents with custom IDs
vector_store.add_documents(documents=documents, ids=["doc1", "doc2"])

# Delete documents
vector_store.delete(ids=["doc1"])

# Search for similar documents
query = "What is the weather forecast for tomorrow?"
results = vector_store.similarity_search(query, k=5)
print(results)

[Document(id='doc2', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'), Document(id='d1378887-3b83-4dfb-acd5-eda454b30cfc', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'), Document(id='a3535eb6-40d0-48f2-ad59-de4bbc15c6c2', metadata={'source': 'news'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')]


## Embedding Models and Text Loaders

In [91]:
fp = "data/constitution.pdf"

loader = PyPDFLoader(fp)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

documents = text_splitter.split_documents(docs)
print(len(documents))
print(documents[4])

138
page_content='among the several States which may be included within 
this Union, according to their respective Numbers, which 
shall be determined by adding to the whole Number of 
free Persons, including those bound to Service for a Term 
of Years, and excluding Indians not taxed, three fifths of 
all other Persons.]* The actual Enumeration shall be made 
within three Years after the first Meeting of the Congress 
of the United States, and within every subsequent Term of' metadata={'source': 'data/constitution.pdf', 'page': 1}


## Vector Store Creation Using Chroma and FAISS

In [92]:
from uuid import uuid4
from langchain_chroma import Chroma
# init vector store
vector_store = Chroma(
    collection_name="constitution",
    embedding_function=embedding_model,
    persist_directory="chroma",
)
uuids = [str(uuid4()) for _ in range(len(documents))]

# Add documents
vector_store.add_documents(documents=documents, ids=uuids)

# Update documents
vector_store.update_documents(
    ids=uuids[:2], documents=[document_1, document_2]
)



### FAISS

In [93]:
from langchain_community.vectorstores import FAISS


################### FAISS ###################
vector_store_faiss = FAISS.from_documents(documents, embedding_model, ids=uuids)

vector_store_faiss.save_local('faiss')

## Similarity search

In [94]:
query = "What is the role of the President of the United States? What are the powers of the President of the United States?"

results = vector_store.similarity_search(query,  filter={"source": "news"})
# results = vector_store.similarity_search(query, k=5)

results

[Document(metadata={'page': 0, 'source': 'news'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'page': 0, 'source': 'news'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'page': 1, 'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'page': 1, 'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.')]

### Similarity search by vector

In [95]:
embedding_vector = embedding_model.embed_query(query)
docs = vector_store_faiss.similarity_search_by_vector(embedding_vector, k=5)

docs

[Document(metadata={'source': 'data/constitution.pdf', 'page': 12}, page_content='to the President of the Senate;-the President of the Senate \nshall, in the presence of the Senate and House of Represen- \ntatives, open all the certificates and the votes shall then be \ncounted;-The person having the greatest number of votes \nfor President, shall be the President, if such number be a \nmajority of the whole number of Electors appointed; and if \nno person have such majority, then from the persons having \nthe highest numbers not exceeding three on the list of those'),
 Document(metadata={'source': 'data/constitution.pdf', 'page': 16}, page_content='shall be elected to the office of President more than once. \nBut this Article shall not apply to any person holding the \noffice of President when this Article was proposed by Con- \ngress, and shall not prevent any person who may be holding \nthe office of President, or acting as President, during the \nterm within which this Article beco

## Asynchronous Vector Store

In [96]:
docs = await vector_store_faiss.asimilarity_search(query)
docs


[Document(metadata={'source': 'data/constitution.pdf', 'page': 12}, page_content='to the President of the Senate;-the President of the Senate \nshall, in the presence of the Senate and House of Represen- \ntatives, open all the certificates and the votes shall then be \ncounted;-The person having the greatest number of votes \nfor President, shall be the President, if such number be a \nmajority of the whole number of Electors appointed; and if \nno person have such majority, then from the persons having \nthe highest numbers not exceeding three on the list of those'),
 Document(metadata={'source': 'data/constitution.pdf', 'page': 16}, page_content='shall be elected to the office of President more than once. \nBut this Article shall not apply to any person holding the \noffice of President when this Article was proposed by Con- \ngress, and shall not prevent any person who may be holding \nthe office of President, or acting as President, during the \nterm within which this Article beco

# Chat with Vector Store

In [97]:
from langchain.prompts import PromptTemplate

llm = ChatOllama(model='llama3.2:1b')

In [98]:
# Perform a vector store similarity search
context_results = vector_store_faiss.similarity_search(query, k=4)

# Create a prompt template and fill it with context
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template="Based on the following context:\n{context}\n\n{query}"
)

# Fill the prompt template with the search results and the query
full_prompt = prompt_template.format(
    context="\n".join([doc.page_content for doc in context_results]),
    query=query
)

# Use the language model to generate a response
response = llm.invoke(full_prompt)
print(response.content)

Based on this context, here's an overview of the roles and powers of the President of the United States:

**Roles:**

1. Commander-in-Chief: The President serves as the commander-in-chief of all branches of the US military.
2. Chief Executable Power: The President has the power to execute the laws of the United States, although some specific responsibilities are delegated to other government officials, such as the Vice President and Congress.
3. Legislator: The President is a co-equal partner with Congress in lawmaking, with the ability to propose legislation and veto it (although Congress can override the veto).
4. Executive Officer: In times of emergency or incapacitation, the Vice President assumes executive power.

**Powers:**

1. Veto Power: The President has the power to veto bills passed by Congress, although Congress can override the veto with a two-thirds majority.
2. Signing Authority: The President has the power to sign bills into law, although they can be vetoed by Congress